# <span style="color:#f37726">04 RKI Feature Engineering</span>

**Ziele des Notebooks:**
- Datensätze für die Prognose vorbereiten
- Trainings- und Testdatensätze für maschinelle Lernmodelle erstellen und exportieren
- Ggf. Attribute skalieren
- Ggf. neue Attribute erstellen

In [1]:
# Benötigte Bibliotheken für dieses Notebook
import json
import numpy as np
import pandas as pd
from pickle import dump
from sklearn.preprocessing import StandardScaler

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Laden der Datensätze als pandas DataFrame
df_train = pd.read_pickle('data/fallzahlen_akkumuliert.pkl')
landkreise_df = pd.read_pickle('data/landkreise_bundesland_id.pkl')
landkreise_series = landkreise_df['Bevölkerung']

data1 = pd.read_csv(r'data/RKI_COVID19_updated.csv')
data2 = pd.read_csv(r'data/cluster_kmeans.csv', delimiter=';', encoding='cp1252')
data3 = pd.read_csv(r'data/cluster_agglo.csv', delimiter=';', encoding='cp1252')

df_test = pd.DataFrame(data1)
cluster_kmeans_df = pd.DataFrame(data2)
cluster_agglo_df = pd.DataFrame(data3)

Zeitreihendaten können aufgrund ihrer Abhängigkeit von der zeitlichen Reihenfolge nicht zufällig in Trainings- und Testdaten eingeteilt werden. Hiermit werden die Zeiträume für die Trainingsperiode bzw. Tesperiode auf den 01. März 2020 bis 31. März 2021 bzw. 01. April 2021 bis 20. Mai 2021 festgesetzt.

## Grundlage Trainingsdaten

In [4]:
from rki_function3 import LagCreator
from rki_function4 import CalendarAdder

df_train = LagCreator(df_train)
df_train = CalendarAdder(df_train)

cluster_kmeans_df.set_index('id', inplace=True, drop=True)
cluster_kmeans_series = cluster_kmeans_df['Cluster']
cluster_agglo_df.set_index('id', inplace=True, drop=True)
cluster_agglo_series = cluster_agglo_df['Cluster']

Idindex = df_train.index.get_level_values('IdLandkreis')
df_train['Bevölkerung'] = landkreise_series.loc[Idindex].values
df_train['kmeans_cluster'] = cluster_kmeans_series.loc[Idindex].values
df_train['agglo_cluster'] = cluster_agglo_series.loc[Idindex].values

display(df_train)

AnzahlFall  AnzahlTodesfall  AnzahlGenesen  \
IdLandkreis Meldedatum                                               
1001        2020-03-01           0                0              0   
            2020-03-02           0                0              0   
            2020-03-03           0                0              0   
            2020-03-04           0                0              0   
            2020-03-05           0                0              0   
...                            ...              ...            ...   
16077       2021-03-27          27                0              0   
            2021-03-28           9                0              0   
            2021-03-29          29                0              0   
            2021-03-30          38                0              0   
            2021-03-31          79                0              3   

                        AnzahlFall_Vortag  AnzahlFall_Vorzweitag  \
IdLandkreis Meldedatum                                             
1001        2020-03-01                  0                      0   
            2020-03-02                  0                      0   
            2020-03-03                  0                      0   
            2020-03-04                  0                      0   
            2020-03-05                  0                      0   
...                                   ...                    ...   
16077       2021-03-27                 47                     26   
            2021-03-28                 27                     47   
            2021-03-29                  9                     27   
            2021-03-30                 29                      9   
            2021-03-31                 38                     29   

                        AnzahlFall_Vordreitag  AnzahlFall_Vorviertag  \
IdLandkreis Meldedatum                                                 
1001        2020-03-01                      0                      0   
            2020-03-02                      0                      0   
            2020-03-03                      0                      0   
            2020-03-04                      0                      0   
            2020-03-05                      0                      0   
...                                       ...                    ...   
16077       2021-03-27                     50                     38   
            2021-03-28                     26                     50   
            2021-03-29                     47                     26   
            2021-03-30                     27                     47   
            2021-03-31                      9                     27   

                        AnzahlFall_Vorfünftag  AnzahlFall_Vorsechstag  \
IdLandkreis Meldedatum                                                  
1001        2020-03-01                      0                       0   
            2020-03-02                      0                       0   
            2020-03-03                      0                       0   
            2020-03-04                      0                       0   
            2020-03-05                      0                       0   
...                                       ...                     ...   
16077       2021-03-27                     16                      23   
            2021-03-28                     38                      16   
            2021-03-29                     50                      38   
            2021-03-30                     26                      50   
            2021-03-31                     47                      26   

                        AnzahlFall_Vorwoche  AnzahlFall_Vortagwoche  Monat  \
IdLandkreis Meldedatum                                                       
1001        2020-03-01                    0                       0      3   
            2020-03-02                    0                       0      3   
            2020-03-03       

## Grundlage Testdaten


In [5]:
from rki_function1 import CountyCasesAggregator
from rki_function2 import DateCasesAggregator

berliner_ids = [11001, 11002, 11003, 11004, 11005, 11006, 11007, 11008, 11009, 11010, 11011, 11012]

df_test = DateCasesAggregator(df_test, pd.to_datetime('2021-03-23'), pd.to_datetime('2021-05-20'))
df_test = CountyCasesAggregator(df_test, berliner_ids, 11000)
df_test = LagCreator(df_test)
df_test = CalendarAdder(df_test)

In [6]:
test_start_time = pd.to_datetime('2021-04-01')
test_end_time = pd.to_datetime('2021-05-20')
test_dates = pd.date_range(test_start_time, test_end_time, freq='D')

df_test = df_test.loc[(slice(None), test_dates), :]

Idindex = df_test.index.get_level_values('IdLandkreis')
df_test['Bevölkerung'] = landkreise_series.loc[Idindex].values
df_test['kmeans_cluster'] = cluster_kmeans_series.loc[Idindex].values
df_test['agglo_cluster'] = cluster_agglo_series.loc[Idindex].values

In [7]:
display(df_test)

AnzahlFall  AnzahlTodesfall  AnzahlGenesen  \
IdLandkreis Meldedatum                                               
1001        2021-04-01          14                0             14   
            2021-04-02          19                0             19   
            2021-04-03          12                0             12   
            2021-04-04           4                0              4   
            2021-04-05           7                0              7   
...                            ...              ...            ...   
16077       2021-05-16           0                0              0   
            2021-05-17           9                0              0   
            2021-05-18          22                0              0   
            2021-05-19           0                0              0   
            2021-05-20           4                0              0   

                        AnzahlFall_Vortag  AnzahlFall_Vorzweitag  \
IdLandkreis Meldedatum                                             
1001        2021-04-01                 15                     10   
            2021-04-02                 14                     15   
            2021-04-03                 19                     14   
            2021-04-04                 12                     19   
            2021-04-05                  4                     12   
...                                   ...                    ...   
16077       2021-05-16                  3                     15   
            2021-05-17                  0                      3   
            2021-05-18                  9                      0   
            2021-05-19                 22                      9   
            2021-05-20                  0                     22   

                        AnzahlFall_Vordreitag  AnzahlFall_Vorviertag  \
IdLandkreis Meldedatum                                                 
1001        2021-04-01                      9                      4   
            2021-04-02                     10                      9   
            2021-04-03                     15                     10   
            2021-04-04                     14                     15   
            2021-04-05                     19                     14   
...                                       ...                    ...   
16077       2021-05-16                      6                     25   
            2021-05-17                     15                      6   
            2021-05-18                      3                     15   
            2021-05-19                      0                      3   
            2021-05-20                      9                      0   

                        AnzahlFall_Vorfünftag  AnzahlFall_Vorsechstag  \
IdLandkreis Meldedatum                                                  
1001        2021-04-01                     11                      24   
            2021-04-02                      4                      11   
            2021-04-03                      9                       4   
            2021-04-04                     10                       9   
            2021-04-05                     15                      10   
...                                       ...                     ...   
16077       2021-05-16                     40                       6   
            2021-05-17                     25                      40   
            2021-05-18                      6                      25   
            2021-05-19                     15                       6   
            2021-05-20                      3                      15   

                        AnzahlFall_Vorwoche  AnzahlFall_Vortagwoche  Monat  \
IdLandkreis Meldedatum                                                       
1001        2021-04-01                   21                      10      4   
            2021-04-02                   24                      21      4   
            2021-04-03       

## Erstellen und Exportieren der modifizierten Trainings- und Testdaten

**1) Naïve / SNaïve**

In [8]:
train_naive = df_train[['AnzahlFall', 'AnzahlFall_Vorwoche']]
test_naive = df_test[['AnzahlFall', 'AnzahlFall_Vorwoche']]

train_naive.to_pickle(r'data/train_naive.pkl')
test_naive.to_pickle(r'data/test_naive.pkl')

**2) SNaïve mit wöchentlicher Trendkomponente**

In [9]:
train_naive_trend = df_train[['AnzahlFall', 'AnzahlFall_Vortag', 'AnzahlFall_Vorwoche', 'AnzahlFall_Vortagwoche']]
test_naive_trend = df_test[['AnzahlFall', 'AnzahlFall_Vortag', 'AnzahlFall_Vorwoche', 'AnzahlFall_Vortagwoche']]

train_naive_trend.to_pickle(r'data/train_naive_trend.pkl')
test_naive_trend.to_pickle(r'data/test_naive_trend.pkl')

**3) Exponentielle Glättung**

In [10]:
train_smoothing = df_train[['AnzahlFall', 'AnzahlFall_Vorwoche']]
test_smoothing = df_test[['AnzahlFall', 'AnzahlFall_Vorwoche']]

train_smoothing.to_pickle(r'data/train_smoothing.pkl')
test_smoothing.to_pickle(r'data/test_smoothing.pkl')

**4) ARIMA**

In [11]:
train_arima = df_train[['AnzahlFall', 'AnzahlFall_Vorwoche']]
test_arima = df_test[['AnzahlFall', 'AnzahlFall_Vorwoche']]

train_arima.to_pickle(r'data/train_arima.pkl')
test_arima.to_pickle(r'data/test_arima.pkl')

**5) MLP ohne Landkreisdaten**

In [12]:
train_mlp = df_train[['AnzahlFall', 'AnzahlFall_Vortag', 'AnzahlFall_Vorzweitag', 'AnzahlFall_Vordreitag', 'AnzahlFall_Vorviertag', 'AnzahlFall_Vorfünftag', 'AnzahlFall_Vorsechstag', 'AnzahlFall_Vorwoche', 'Wochentag']]
test_mlp = df_test[['AnzahlFall', 'AnzahlFall_Vortag', 'AnzahlFall_Vorzweitag', 'AnzahlFall_Vordreitag', 'AnzahlFall_Vorviertag', 'AnzahlFall_Vorfünftag', 'AnzahlFall_Vorsechstag', 'AnzahlFall_Vorwoche', 'Wochentag']]


# Tage mit sehr niedrigen Fallzahlen entfernen (am Anfang der Pandemie)
train_mlp = train_mlp[train_mlp['AnzahlFall'] > 0]


# Nicht genutzte Attribute entfernen
train_mlp.reset_index(inplace=True, drop=False)
train_mlp.drop('IdLandkreis', axis=1, inplace=True)
train_mlp.drop('Meldedatum', axis=1, inplace=True)

test_mlp.reset_index(inplace=True, drop=False)

# 'Wochentag' als Boolsche Werte
weekdays = ['Montag', 'Dienstag', 'Mittwoch', 'Donnerstag', 'Freitag', 'Samstag', 'Sonntag']
weekdays_dict = dict(zip(train_mlp['Wochentag'].unique(), weekdays))

train_mlp['Wochentag'] = train_mlp['Wochentag'].replace(weekdays_dict)
train_mlp = pd.concat([train_mlp, train_mlp['Wochentag'].str.get_dummies()], axis=1)
train_mlp.drop('Wochentag', axis=1, inplace=True)

test_mlp['Wochentag'] = test_mlp['Wochentag'].replace(weekdays_dict)
test_mlp = pd.concat([test_mlp, test_mlp['Wochentag'].str.get_dummies()], axis=1)
test_mlp.drop('Wochentag', axis=1, inplace=True)


train_mlp.to_pickle(r'data/train_mlp.pkl')
test_mlp.to_pickle(r'data/test_mlp.pkl')

**6) MLP mit den Ergebnissen des Clusterings**

In [13]:
train_mlp_mit = df_train[['AnzahlFall', 'AnzahlFall_Vortag', 'AnzahlFall_Vorzweitag', 'AnzahlFall_Vordreitag', 'AnzahlFall_Vorviertag', 'AnzahlFall_Vorfünftag', 'AnzahlFall_Vorsechstag', 'AnzahlFall_Vorwoche', 'Wochentag', 'kmeans_cluster', 'agglo_cluster']]
test_mlp_mit = df_test[['AnzahlFall', 'AnzahlFall_Vortag', 'AnzahlFall_Vorzweitag', 'AnzahlFall_Vordreitag', 'AnzahlFall_Vorviertag', 'AnzahlFall_Vorfünftag', 'AnzahlFall_Vorsechstag', 'AnzahlFall_Vorwoche', 'Wochentag', 'kmeans_cluster', 'agglo_cluster']]


# Tage mit sehr niedrigen Fallzahlen entfernen (am Anfang der Pandemie)
train_mlp_mit = train_mlp_mit[train_mlp_mit['AnzahlFall'] > 0]


# Nicht genutzte Attribute entfernen
train_mlp_mit.reset_index(inplace=True, drop=False)
train_mlp_mit.drop('IdLandkreis', axis=1, inplace=True)
train_mlp_mit.drop('Meldedatum', axis=1, inplace=True)

test_mlp_mit.reset_index(inplace=True, drop=False)


# 'Wochentag' als Boolsche Werte
weekdays = ['Montag', 'Dienstag', 'Mittwoch', 'Donnerstag', 'Freitag', 'Samstag', 'Sonntag']
weekdays_dict = dict(zip(train_mlp_mit['Wochentag'].unique(), weekdays))

train_mlp_mit['Wochentag'] = train_mlp_mit['Wochentag'].replace(weekdays_dict)
train_mlp_mit = pd.concat([train_mlp_mit, train_mlp_mit['Wochentag'].str.get_dummies()], axis=1)
train_mlp_mit.drop('Wochentag', axis=1, inplace=True)

test_mlp_mit['Wochentag'] = test_mlp_mit['Wochentag'].replace(weekdays_dict)
test_mlp_mit = pd.concat([test_mlp_mit, test_mlp_mit['Wochentag'].str.get_dummies()], axis=1)
test_mlp_mit.drop('Wochentag', axis=1, inplace=True)


# Clustering One-Hot Encoding
one_hot_kmeans_train = pd.get_dummies(train_mlp_mit['kmeans_cluster'])
one_hot_kmeans_train.columns = ['is_kmeans_cluster_' + str(col) for col in one_hot_kmeans_train.columns]
one_hot_kmeans_test = pd.get_dummies(test_mlp_mit['kmeans_cluster'])
one_hot_kmeans_test.columns = ['is_kmeans_cluster_' + str(col) for col in one_hot_kmeans_test.columns]


one_hot_agglo_train = pd.get_dummies(train_mlp_mit['agglo_cluster'])
one_hot_agglo_train.columns = ['is_agglo_cluster_' + str(col) for col in one_hot_agglo_train.columns]
one_hot_agglo_test = pd.get_dummies(test_mlp_mit['agglo_cluster'])
one_hot_agglo_test.columns = ['is_agglo_cluster_' + str(col) for col in one_hot_agglo_test.columns]

train_mlp_mit = train_mlp_mit.join(one_hot_kmeans_train)
train_mlp_mit = train_mlp_mit.join(one_hot_agglo_train)
train_mlp_mit.drop('kmeans_cluster', axis=1, inplace=True)
train_mlp_mit.drop('agglo_cluster', axis=1, inplace=True)

test_mlp_mit = test_mlp_mit.join(one_hot_kmeans_test)
test_mlp_mit = test_mlp_mit.join(one_hot_agglo_test)
test_mlp_mit.drop('kmeans_cluster', axis=1, inplace=True)
test_mlp_mit.drop('agglo_cluster', axis=1, inplace=True)


train_mlp_mit.to_pickle(r'data/train_mlp_mit.pkl')
test_mlp_mit.to_pickle(r'data/test_mlp_mit.pkl')

**7) RNN**

In [14]:
train_rnn = df_train[['AnzahlFall', 'AnzahlFall_Vortag', 'AnzahlFall_Vorzweitag', 'AnzahlFall_Vordreitag', 'AnzahlFall_Vorviertag', 'AnzahlFall_Vorfünftag', 'AnzahlFall_Vorsechstag', 'AnzahlFall_Vorwoche']]
test_rnn = df_test[['AnzahlFall', 'AnzahlFall_Vortag', 'AnzahlFall_Vorzweitag', 'AnzahlFall_Vordreitag', 'AnzahlFall_Vorviertag', 'AnzahlFall_Vorfünftag', 'AnzahlFall_Vorsechstag', 'AnzahlFall_Vorwoche']]


# MinMax Skalierung Vorbereiten
fallzahlen_minmax = {}
kreis_ids = df_train.index.get_level_values(0).unique()



for kreis_id in kreis_ids:
    
    # Minimum und Maximum extrahieren
    kreis_df = df_train.xs(kreis_id)
    kreis_min = kreis_df['AnzahlFall'].min()
    kreis_max = kreis_df['AnzahlFall'].max()
    fallzahlen_minmax[kreis_id] =[kreis_min, kreis_max]

    
train_rnn.reset_index(inplace=True, drop=False)
test_rnn.reset_index(inplace=True, drop=False)


for kreis_id in kreis_ids:
    
    kreis_min = fallzahlen_minmax[kreis_id][0]
    kreis_max = fallzahlen_minmax[kreis_id][1]
    
    kreis_train_df = train_rnn.loc[train_rnn['IdLandkreis'] == kreis_id]
    kreis_test_df = test_rnn.loc[test_rnn['IdLandkreis'] == kreis_id]
     
    kreis_train_df[['AnzahlFall', 'AnzahlFall_Vortag', 'AnzahlFall_Vorzweitag', 'AnzahlFall_Vordreitag', 'AnzahlFall_Vorviertag', 'AnzahlFall_Vorfünftag', 'AnzahlFall_Vorsechstag', 'AnzahlFall_Vorwoche']] = kreis_train_df[['AnzahlFall', 'AnzahlFall_Vortag', 'AnzahlFall_Vorzweitag', 'AnzahlFall_Vordreitag', 'AnzahlFall_Vorviertag', 'AnzahlFall_Vorfünftag', 'AnzahlFall_Vorsechstag', 'AnzahlFall_Vorwoche']].apply(lambda x: (x-kreis_min)/(kreis_max-kreis_min), axis=1)
    kreis_test_df[['AnzahlFall', 'AnzahlFall_Vortag', 'AnzahlFall_Vorzweitag', 'AnzahlFall_Vordreitag', 'AnzahlFall_Vorviertag', 'AnzahlFall_Vorfünftag', 'AnzahlFall_Vorsechstag', 'AnzahlFall_Vorwoche']] = kreis_test_df[['AnzahlFall', 'AnzahlFall_Vortag', 'AnzahlFall_Vorzweitag', 'AnzahlFall_Vordreitag', 'AnzahlFall_Vorviertag', 'AnzahlFall_Vorfünftag', 'AnzahlFall_Vorsechstag', 'AnzahlFall_Vorwoche']].apply(lambda x: (x-kreis_min)/(kreis_max-kreis_min), axis=1)
    
    train_rnn.loc[train_rnn['IdLandkreis'] == kreis_id] = kreis_train_df
    test_rnn.loc[test_rnn['IdLandkreis'] == kreis_id] = kreis_test_df
    

train_rnn = train_rnn.groupby(['IdLandkreis', 'Meldedatum']).last()
test_rnn = test_rnn.groupby(['IdLandkreis', 'Meldedatum']).last()


train_rnn.to_pickle(r'data/train_rnn.pkl')
test_rnn.to_pickle(r'data/test_rnn.pkl')

with open("data/fallzahlen_minmax.json", "w") as outfile: 
    json.dump(fallzahlen_minmax, outfile)

Die erstellten Trainingsdatensätze sollen im nächsten Notebook für das Training der maschinellen Lernmodelle verwendet werden.